# inforet 2024 3

### bm 25 tests and query expansion by synonyms

In [ ]:
!pip install rank_bm25 spacy Sense2Vec
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from rank_bm25 import BM25Okapi
import spacy
from sense2vec import Sense2Vec
tqdm.pandas()

In [ ]:
# this turns on the autotimer, so that every cell has a timing information below
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
# stop using:
# %unload_ext autotime

## getting the best combinations from last time and writing them into files

In [ ]:
origdocs = pd.read_csv('our.msmarco.docs.tsv',sep='\t',usecols=[1,2,3])
origdocs['title'].fillna('-', inplace=True)
origdocs['body'].fillna('-', inplace=True)
origdocs

In [ ]:
docs = pd.DataFrame(columns = ['docid', 'text'])
docs['docid']=origdocs.docid
docs['text']=origdocs.title+' '+origdocs.body
docs

In [ ]:
del origdocs # saving memory

In [ ]:
docs.to_csv('our.text.msmarco.docs.tsv',sep='\t', columns=['docid','text'])

#### and now the pre-tokenization for bm25

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode')
tokenized_corpus = docs.text.progress_apply(vectorizer.build_analyzer())
#docs['docid'].to_frame().join(tokenized_corpus)

In [ ]:
type(tokenized_corpus)

## reading back in just for checking the files - or for restarting here

In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from rank_bm25 import BM25Okapi

tqdm.pandas()

In [ ]:
# this is a different doc, no longer distinguishing title and body
docs = pd.read_csv('our.text.msmarco.docs.tsv',sep='\t',usecols=[1,2]) 
docs

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode')
tokenized_corpus = docs.text.progress_apply(vectorizer.build_analyzer())

In [ ]:
# use only col 1 if you have memory problems and do BM25 only
queries = pd.read_csv('our.msmarco.queries.tsv',sep='\t',usecols=[1,2]) 
training_queries=queries.iloc[:500]
testing_queries=queries.iloc[500:]
training_queries

In [ ]:
gold = pd.read_csv('our.msmarco.gold.tsv',sep='\t',usecols=[1,3,4,5])
gold

# redoing the vectorization for my two best results

### 🚧 todo:
### use TfidfVectorizer, BM25Okapi, and our own BM25 function
to measure whether there are significant differences.


In [ ]:
def pAt10(qid):
    query = queries[queries.qid==qid]['query']
    qv = vectorizer.transform(query)
    xqv = X*qv.T
    pred10i = np.argpartition(xqv.A.flat, -10)[-10:]
    intersection = np.intersect1d(docs.loc[pred10i].docid,gold[gold.qid==qid].docid)
    return len(intersection)/10

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, strip_accents='unicode')
X = vectorizer.fit_transform(docs.text)
print(len(vectorizer.get_feature_names()),'features, for example',vectorizer.get_feature_names()[44444:44449])
tfidfresults = training_queries.qid.progress_apply(pAt10)
tfidfresults.mean()

In [ ]:
def pAt10Bm25(qid):
    tquery = queries[queries.qid==qid]['query'].apply(vectorizer.build_analyzer())
    doc_scores = bm25.get_scores(tquery.tolist()[0])
    pred10i = np.argpartition(doc_scores, -10)[-10:]
    intersection = np.intersect1d(docs.loc[pred10i].docid,gold[gold.qid==qid].docid)
    return len(intersection)/10

In [ ]:
bm25 = BM25Okapi(tokenized_corpus)
bm25results = training_queries.qid.progress_apply(pAt10Bm25)
bm25results.mean()

# 🔎 manual error mining
- let's look at where things go wrong

### 🚧 todo:
- what's the lowest p@10 we got
- what's the 10 questions that got the worst score, from worst to slightly better?


In [ ]:
...

In [ ]:
worst10bm25i = ...
worst10bm25i

In [ ]:
training_queries.loc[worst10bm25i]

### 🚧 todo:
- write a function showDoc that takes qid, rank, and predicted as parameters
    - if predicted=True, shows the predicted doc of rank rank to the query qid
    - if predicted=False, shows the gold doc
    - prints the first 999 characters of the texts
- for the worst query
    - look at the 10 best gold vs 10 best predicted 
    - hypothetize why the results are so bad for the worst query

In [ ]:
def showDoc(qid,rank,predicted=False):
    if predicted:
        ...
    else:
        ...
showDoc(729561,7)
showDoc(729561,7, predicted=True)

### 🚧 todo: can we characterize these difficult cases?
- do they have specicific problems?
- do we know when we are doing badly?
    - are the distances between query vector and the best documents bigger than average?
    

# 🚀 spacy

- look at https://github.com/explosion/sense2vec/blob/master/README.md

In [ ]:
nlp = spacy.load('en_core_web_lg') # or the smaller md model!!!

### 🚧 todo:
- explain what's going on here:

In [ ]:
sent1 = nlp("I am happy")
sent2 = nlp("I am sad")
sent3 = nlp("I am joyful")
sent1.similarity(sent2), sent1.similarity(sent3)

### let's try sense2vec

- depending on your machine, download one of the two versions of sense2vec from https://github.com/explosion/sense2vec/blob/master/README.md
  - s2v_reddit_2019_lg 	4 GB 	Reddit comments 2019 (01-07) 	part 1, part 2, part 3
      - cat s2v_reddit_2019_lg.tar.gz.* > s2v_reddit_2019_lg.tar.gz
  - s2v_reddit_2015_md 	573 MB 	Reddit comments 2015 	part 1
- unzip
- try it, and understand what's going on:


In [ ]:
s2v = Sense2Vec().from_disk("./s2v_reddit_2019_lg")


In [ ]:
seeds = "natural language processing, machine learning, artificial intelligence".split(',')
seed_keys = [s2v.get_best_sense(seed.strip()) for seed in seeds]
seed_keys

In [ ]:
most_similar = s2v.most_similar(seed_keys, n=10)
most_similar

### 🚧 todo: what is it that you couldn't do in Word2Vec?
- just one line of answer.
- answer: ...

- most_similar is very slow. check this to speed things up (optional): https://towardsdatascience.com/how-to-build-a-fast-most-similar-words-method-in-spacy-32ed104fe498
### 🚧 todo:
- try also the following functions: 
    - similarity, get_other_senses, get_freq, s2v[query]


In [ ]:
...

### 🚧 todo:
- try whether expanding your query by adding similar terms to the 10 worst queries improves the results


In [ ]:
...

### 🚧 todo:
- try misspelling a word and see whether you can fix that with sense2vec


### 🚧 todo:
- try embeddings for a few queries (all would take to long except if you have a GPU)
    - are the gold top 10 similar to the query itself?
    - check whether the gold top 10 answers for our most difficult question are really closer to the question than the currently predicted top10
         - how to get every doc as a vector: 
             - https://spacy.io/api/doc#vector "A real-valued meaning representation. Defaults to an average of the token vectors."
        - every doc has a similarity function taking another doc as argument: 
            - https://spacy.io/api/doc#similarity

In [5]:
...

Ellipsis

In [ ]:
# not necessary but if you want to include your big s2v file
# combining spacy and sense2vec:
nlp = spacy.load("en_core_web_sm") # or whichever you downloaded
s2v = nlp.add_pipe("sense2vec")
s2v.from_disk("./s2v_reddit_2015_md") # or whichever you downloaded